In [64]:
import pandas as pd
import numpy as np
import time
from math import log
from sklearn.preprocessing import StandardScaler
from imutils import paths
import cv2
import matplotlib.pyplot as plt
from xml.dom import minidom
import tensorflow as tf
from tensorflow.keras.models import Sequential
import tensorflow.keras.layers as tfl
from tensorflow.keras.losses import CategoricalCrossentropy , mae,mse
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.preprocessing import image_dataset_from_directory as load_dataset
import os
import matplotlib.patches as patches
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

In [2]:
directory="./car_plate_fa/validation"
i=500
resized_width=400
resized_height=400
depth=3
n_y_features=4
BATCH_SIZE = 64
IMG_SIZE = (resized_height, resized_width)
split_size=0.1 

In [44]:
%run common.ipynb

In [35]:
# persian plates
y=np.zeros((i,4))
file_paths=[]
fig = plt.figure(figsize=(30, 30))
for carNumber, carPath in enumerate(getPersianPlates(end=i)):
    file_paths.append(carPath)
    print("\r",carNumber,end="")
    img=cv2.imread(carPath)
    original_width=img.shape[1]
    original_height=img.shape[0]
    w_ratio,h_ratio=compute_ratio(original_width,original_height,resized_width,resized_height)
    xmlPath=faImgPath +"_labels" +"/" + carPath.split(os.path.sep)[-1].split(".")[0] +".xml"
    xmlFile=minidom.parse(xmlPath)
    xmin=int (float(xmlFile.getElementsByTagName("xmin")[0].firstChild.data))
    ymin=int (float(xmlFile.getElementsByTagName("ymin")[0].firstChild.data)) 
    xmax=int (float(xmlFile.getElementsByTagName("xmax")[0].firstChild.data)) 
    ymax=int (float(xmlFile.getElementsByTagName("ymax")[0].firstChild.data))
    

    xmin=int( xmin * w_ratio )
    ymin=int( ymin * h_ratio )
    xmax=int( xmax * w_ratio )
    ymax=int( ymax * h_ratio )

    y[carNumber][0]=xmin
    y[carNumber][1]=ymin
    y[carNumber][2]=xmax
    y[carNumber][3]=ymax
   
y=y.astype(np.int32)


 499

<Figure size 3000x3000 with 0 Axes>

In [82]:
# get dataset and split
train_size=int(len(y)*(1-split_size))
dataset = tf.data.Dataset.from_tensor_slices((file_paths, y))

def load_and_preprocess_image(file_path, label):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image,[ resized_height,resized_width])
    return image, label

dataset = dataset.map(load_and_preprocess_image)
train_dataset=dataset.take(train_size).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset=dataset.skip(train_size).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
# dataset=dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
# get_size=tf.data.experimental.cardinality
# get_size(train_dataset),get_size(val_dataset)

In [84]:
def preprocess_image(file_path):
    image = cv2.imread(file_path, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (resized_width, resized_height))
    return image
def data_generator():
    for file_path, label in zip(file_paths, y):
        image = preprocess_image(file_path)
        yield image, label



dataset = tf.data.Dataset.from_generator(
    generator=data_generator,
    output_types=(tf.float32, tf.int32),  
    output_shapes=(
        tf.TensorShape([resized_height, resized_width, 3]),
        tf.TensorShape([4])  
    )
)

train_dataset = dataset.take(train_size).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = dataset.skip(train_size).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)


In [45]:
train_dataset,val_dataset =get_splitted_dataset(end=i,BATCH_SIZE=BATCH_SIZE,split_size=0.1)
train_dataset,val_dataset

(<PrefetchDataset element_spec=(TensorSpec(shape=(None, 400, 400, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.int32, name=None))>,
 <PrefetchDataset element_spec=(TensorSpec(shape=(None, 400, 400, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.int32, name=None))>)

In [55]:
# test dataset

for images,labels in train_dataset.take(1):
    j=30
    img=images[j].numpy()
    img=cv2.rectangle(img,(labels[j][0].numpy().astype(np.int32),
                           labels[j][1].numpy().astype(np.int32)),
                      (labels[j][2].numpy().astype(np.int32),
                       labels[j][3].numpy().astype(np.int32)),
                      (255,0,0))
    
    img=cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
    cv2.imshow("original",img.astype(np.uint8))
    cv2.waitKey()
    cv2.destroyAllWindows()


In [51]:
model=plate_boundary_model(resized_height,resized_width)

In [56]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 400, 400, 3)]     0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 400, 400, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 400, 400, 3)      0         
 a)                                                              
                                                                 
 inception_resnet_v2 (Functi  (None, 11, 11, 1536)     54336736  
 onal)                                                           
                                                                 
 flatten_3 (Flatten)         (None, 185856)            0         
                                                           

In [60]:
base_learning_rate = 0.001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=mse,
              metrics=['accuracy'])

In [58]:
save_path="./car_plate_weights/result.hdf5"
checkpoint = ModelCheckpoint(save_path, monitor="loss",
	save_best_only=True, verbose=1)

callbacks = [checkpoint]

In [61]:

initial_epochs = 70

history = model.fit(train_dataset,
                    # validation_data=val_dataset,
                    epochs=initial_epochs,
                    callbacks=callbacks
                    )

Epoch 1/70
      8/Unknown - 21s 1s/step - loss: 1548.7256 - accuracy: 0.9311
Epoch 1: loss improved from 1689.52966 to 1548.72559, saving model to ./car_plate_weights\result.hdf5
8/8 [==============================] - 29s 3s/step - loss: 1548.7256 - accuracy: 0.9311
Epoch 2/70
8/8 [==============================] - ETA: 0s - loss: 1392.6407 - accuracy: 0.9178
Epoch 2: loss improved from 1548.72559 to 1392.64075, saving model to ./car_plate_weights\result.hdf5
8/8 [==============================] - 19s 2s/step - loss: 1392.6407 - accuracy: 0.9178
Epoch 3/70
8/8 [==============================] - ETA: 0s - loss: 1130.5930 - accuracy: 0.9467
Epoch 3: loss improved from 1392.64075 to 1130.59302, saving model to ./car_plate_weights\result.hdf5
8/8 [==============================] - 19s 2s/step - loss: 1130.5930 - accuracy: 0.9467
Epoch 4/70
8/8 [==============================] - ETA: 0s - loss: 1041.0067 - accuracy: 0.9400
Epoch 4: loss improved from 1130.59302 to 1041.00671, saving model 

In [65]:
loaded_model=load_model(save_path)

In [92]:
loaded_model.evaluate(train_dataset)

8/8 [==============================] - 13s 1s/step - loss: 327.8635 - accuracy: 0.9444


[327.8634948730469, 0.9444444179534912]

In [69]:
# test dataset
for images,labels in train_dataset.take(3):
    j=15
    img=images[j].numpy()
    preds=loaded_model.predict(images).astype(np.int32)
    labels=labels.numpy().astype(np.int32)
    img=cv2.rectangle(img,preds[j][0:2],preds[j][2:],(255,127,0))
    img=cv2.rectangle(img,labels[j][0:2],labels[j][2:],(0,255,0))
    img=cv2.cvtColor(img,cv2.COLOR_RGB2BGR).astype(np.uint8)
    cv2.imshow("win",img)
    cv2.waitKey()
    cv2.destroyAllWindows()

2/2 [==============================] - 1s 525ms/step
